In [1]:
# Function for generating histogram
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlt
import datetime 

In [2]:
# This is to have a broader function that can get the data in the first place. (Datetime as well.)
def getData(year, direct, ASC, start_day, days, cols,datatype):
    for i in range(0, days):
        day = i + start_day
        year_str = '0.' + str(year)
        # File 
        if (day < 10):
            holdstr = 'pton00' + str(day) + year_str 
        elif (day < 100):
            holdstr ='pton0' + str(day) + year_str
        else: 
            holdstr = 'pton' + str(day) + year_str
        path = direct + holdstr + ASC
        if (day == start_day):
            data = np.array(np.loadtxt(path, dtype = datatype, skiprows = 1, usecols = cols))
        else: 
            hold = np.array(np.loadtxt(path, dtype = datatype, skiprows = 1, usecols = cols))
            data = np.concatenate((data, hold))
    return data

In [3]:
# Puts in the data that you have and returns precip events, nonprecip events, and total precip per precip event
def preciphist(data, minute, length, prec_duration, nonprec_duration):
    precipevent = np.zeros(length)
    nonprecip = np.zeros(length)
    preciptotal = np.zeros(length) # This is not intensity, but total precip per corresponding event.
    # This is to represent indices for the corresponding array from above
    prec = 0 # This represents both the precip events and total precip from precip events indices.
    nonprec = 0
    k = 0
    holdprec = 0 # These next three variables will hold the values, as we try to expand to include variable duration! 
    holdnonprec = 0
    holdtotal = 0
    # 
    Total_precip = 0; # Total precipitation measured by instrument 
    while (k < minute):
        hold = data[k]
        # If there is no precipitation at a certain minute
        if (hold == 0):
            holdnonprec += 1
            # How to deal with previous precipitation event
            if (holdprec >= prec_duration):
                precipevent[prec] = holdprec
                preciptotal[prec] = holdtotal
                prec += 1
                holdprec = 0
                holdtotal = 0
                # How to deal with a short precip event between nonprecip events
            elif (holdprec < prec_duration):
                holdnonprec += holdprec
                holdprec = 0
        #if (k >= 1):
         #   holdbefore = data[k-1]
          #  if (hold != holdbefore and hold == 0 and holdbefore > 0):
           #     prec += 1
        
        # Condition where we have precipitation    
        if (hold > 0):
            Total_precip += hold
            holdprec += 1
            holdtotal += hold
            # How to deal with previous non-precipitation event 
            if (holdnonprec >= nonprec_duration):
                nonprecip[nonprec] = holdnonprec
                nonprec += 1
                holdnonprec = 0
            # How to deal with a short nonprecip event between precip events
            elif (holdnonprec < nonprec_duration):
                holdprec += holdnonprec
                holdnonprec = 0
            # How to account for new precip events
            #if (k >= 1):
             #   holdbefore = data[k-1]
              #  # Don't iterate prec if it is the first precip event
               # if (hold != holdbefore and hold > 0 and holdbefore == 0):
                #    nonprec += 1
        k = k+1
    return precipevent, nonprecip, preciptotal, Total_precip

In [5]:
def hist_plots(array,bin_spec, range_tup, xlabel, ylabel, title, color, eqn, percent):
    plt.figure(figsize = (10,8))
    plt.hist(array[array != 0], bins = bin_spec, range= range_tup, log = log, color = color)
    plt.xlabel(xlabel, fontsize = 14)
    plt.ylabel(ylabel, fontsize = 14)
    plt.title(title, fontsize = 20)
    plt.text(1,1, 'Histogram has ' + percent + ' of total precipitation', horizontalalignment = 'center',
            verticalalignment = 'center')
    plt.text(0.8,1, eqn,horizontalalignment = 'center',
            verticalalignment = 'center' )

In [8]:
direct = 'C:/Users/tyron/Documents/Geoscience/weather/'
ASC = '__ASC_ASCIIIn.mrk'
data2019 = getData(19, direct, ASC, 1, 365,6, float)


In [11]:
precip, nonprecip, preciptotal, totalprep = preciphist(data2019, len(data2019), 20000, 5, 1) 

In [12]:
count0 = 0
count1 = 0
for i in range(0, 20000):
    hold0 = precip[i]
    hold1 = nonprecip[i]
    if (hold0 > 0):
        count0 +=1
    if (hold1 > 0):
        count1 +=1
print(count0)
print(count1)

644
3122
